In [1]:
%pip install open_clip_torch

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
from PIL import Image
import open_clip

#  ('EVA01-g-14', 'laion400m_s11b_b41k'), # g = global
#  ('EVA01-g-14-plus', 'merged2b_s11b_b114k'), # g = global
#  ('EVA02-B-16', 'merged2b_s8b_b131k'), # B = big
#  ('EVA02-L-14', 'merged2b_s4b_b131k'), # L = large
#  ('EVA02-L-14-336', 'merged2b_s6b_b61k'), # L = large
#  ('EVA02-E-14', 'laion2b_s4b_b115k'), # E = efficient
#  ('EVA02-E-14-plus', 'laion2b_s9b_b144k'), # E = efficient


model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
model.eval()  # model in train mode by default, impacts some models with BatchNorm or stochastic depth active
tokenizer = open_clip.get_tokenizer('ViT-B-32')

image = preprocess(Image.open("docs/CLIP.png")).unsqueeze(0)
text = tokenizer(["a diagram", "a dog", "a cat"])

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)  # prints: [[1., 0., 0.]]